# **Квазиньютоновские методы**

✍ В предыдущем юните мы рассмотрели метод Ньютона. В отличие от градиентного спуска, метод Ньютона использует на каждой итерации не только градиент, но и матрицу Гессе. Это обеспечивает более быструю сходимость к минимуму, но в то же время это приводит к слишком большим вычислительным затратам. В данном юните мы рассмотрим класс методов, в которых решается эта проблема, — класс **квазиньютоновских методов**.

Напомним, что в методе Ньютона мы обновляем точку на каждой итерации в соответствии со следующим правилом:

![](data/12.PNG)

На каждом шаге здесь вычисляется гессиан, а также его обратная матрица.

В квазиньютоновских методах вместо вычисления гессиана мы просто аппроксимируем его матрицей, которая обновляется от итерации к итерации с использованием информации, вычисленной на предыдущих шагах. Так как вместо вычисления большого количества новых величин мы используем найденные ранее значения, квазиньютоновский алгоритм тратит гораздо меньше времени и вычислительных ресурсов.

Формально это описывается следующим образом:

![](data/13.PNG)

В данном случае вместо обратного гессиана появляется матрица Hk, которая строится таким образом, чтобы максимально точно аппроксимировать настоящий обратный гессиан.

Математически это записывается так:

![](data/14.PNG)

Здесь имеется в виду, что разница между матрицей в квазиньютоновском методе и обратным гессианом стремится к нулю.

Эта матрица обновляется на каждом шаге, и для этого существуют разные способы. Для каждого из способов есть своя модификация квазиньютоновского метода. Эти способы объединены ограничением: **процесс обновления матрицы** должен быть достаточно эффективным и **не должен требовать вычислений гессиана**. То есть, по сути, на каждом шаге мы должны получать информацию о гессиане, не находя непосредственно сам гессиан.

Если вас интересует математическая сторона обновления и аппроксимации матрицы, прочитайте эту [**статью**](http://www.machinelearning.ru/wiki/images/6/65/MOMO17_Seminar6.pdf). В силу того, что понимание этой части метода требует очень серьёзной математической подготовки, мы опустим её. Однако можем заверить вас, что для успешного использования алгоритма и его понимания знание всех математических выводов не требуется.

**Три самые популярные схемы аппроксимации:**

* симметричная коррекция ранга 1 (SR1);
* схема Дэвидона — Флетчера — Пауэлла (DFP);
* схема Бройдена — Флетчера — Гольдфарба — Шанно (BFGS).

Последняя схема (BFGS) самая известная, стабильная и считается наиболее эффективной. На ней мы и остановимся. Своё название она получила из первых букв фамилий создателей и исследователей данной схемы: Чарли Джорджа Бройдена, Роджера Флетчера, Дональда Гольдфарба и Дэвида Шанно.

У этой схемы есть две известных вариации:

* L-BFGS;
* L-BFGS-B.

Обе этих вариации необходимы в случае большого количества переменных для экономии памяти (так как во время их реализации хранится ограниченное количество информации). По сути, они работают одинаково, и L-BFGS-B является лишь улучшенной версией L-BFGS для работы с ограничениями.

Метод BFGS очень устойчив и на данный момент считается одним из наиболее эффективных. Поэтому, если, например, применить функцию optimize без указания метода в библиотеке SciPy, то по умолчанию будет использоваться именно BFGS либо одна из его модификаций, указанных выше. Также данный метод используется в библиотеке sklearn при решении задачи логистической регрессии.
***

Рассмотрим **алгоритм применения этого метода**. Постарайтесь понять последовательность действий и основной принцип.

                                                    1
Реализация алгоритма начинается с того, что мы задаём начальную точку x0, выбираем точность алгоритма, а также изначальную аппроксимацию для обратного гессиана функции. 

Здесь как раз таится главная проблема: нет никакого универсального рецепта для выбора этого приближения. Можно поступить по-разному: использовать гессиан, вычисленный в изначальной точке, взять единичную матрицу (такая настройка стоит по умолчанию в некоторых функциях библиотек Python) или другую матрицу, если она невырождена и хорошо обусловлена.

                                                    2
Когда мы определили, откуда будем начинать, необходимо понять, как попасть в следующую точку. Поэтому на втором шаге мы вычисляем направление для поиска следующей точки:

![](data/15.PNG)

                                                    3
Далее находим следующую точку, используя соотношение:

![](data/16.PNG)

Здесь важным вопросом является нахождение коэффициента k, который регулирует шаг. Его подбирают линейным поиском в соответствии с условиями, о которых можно подробно прочитать [**здесь**](https://en.wikipedia.org/wiki/Wolfe_conditions).

![](data/17.PNG)

                                                    4
На следующем шаге необходимо определить два следующих вектора:

![](data/18.PNG)

Здесь sk — шаг алгоритма, a yk — изменение градиента на данной итерации. Они не требовались для перехода в следующую точку и нужны строго для того, чтобы найти следующее приближение обратной матрицы гессиана.

                                                    5
После того как мы их нашли, обновляем гессиан, руководствуясь следующей формулой:

![](data/19.PNG)

Не стоит её пугаться. Как уже было сказано выше, эта формула —  результат очень серьёзных и длительных математических исследований. Её не требуется знать наизусть или уметь реализовывать вручную. Однако для полноты повествования мы не можем не привести её.

В данной формуле за I обозначена единичная матрица, sk и yk мы вычислили на предыдущем шаге, а k вычисляется следующим образом:

![](data/20.PNG)

Алгоритм довольно сложный, поэтому давайте рассмотрим пример ↓

Сразу оговоримся, что несколько шагов в этом алгоритме мы приведём без ручных расчётов (например, нахождение следующей аппроксимации гессиана) в силу их высокой сложности. Постарайтесь сильнее всего сконцентрироваться на шагах решения и понять логику работы алгоритма и последовательность действий. Мы начнём реализовывать алгоритм «вручную», а затем вы сможете самостоятельно завершить решение задачи с использованием Python (разумеется, далее будет указано, как это сделать). К сожалению, серьёзные и эффективные методы настолько сложны, что решать с их помощью задачи, используя только лист бумаги, ручку и калькулятор (как мы могли это делать, например, с методом Лагранжа), уже невозможно.
***

**ПРИМЕР**

![](data/21.PNG)
![](data/22.PNG)
![](data/23.PNG)
![](data/24.PNG)

***

Разумеется, при решении прикладных задач не понадобится делать ничего подобного, ведь мы умеем программировать и знаем, что в библиотеках Python есть все необходимые методы.

Давайте рассмотрим, как с помощью функций Python мы сможем применить квазиньютоновские методы для оптимизации функции x** 2 + y**2.

Подгрузим необходимые библиотеки:

In [1]:
import numpy as np
from scipy.optimize import minimize

# Определим функцию, которую будем оптимизировать.
# Вместо отдельных x и y можно взять координаты единого вектора:
def func(x):
    return x[0]**2.0 + x[1]**2.0

# Теперь определим градиент для функции:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

# Зададим начальную точку:
x_0 = [1.0, 1.0]

# Определим алгоритм:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

#Выведем результаты:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы получили, что минимум функции достигается в точке (0,0). Значение функции в этой точке также равно нулю.

Можно повторить то же самое с вариацией L-BFGS-B:

In [2]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Результат будет тем же самым.

→ Иногда количество итераций у двух модификаций различается, но ответ совпадает. Бывает также, что одна из вариаций может не сойтись, а другая — достичь экстремума, поэтому советуем не воспринимать их как взаимозаменяемые алгоритмы. На практике лучше пробовать разные варианты: если у вас не сошёлся алгоритм BFGS, можно попробовать L-BFGS-B, и наоборот. Также можно экспериментировать одновременно с обоими алгоритмами, чтобы выбрать тот, который будет сходиться для функции за меньшее число итераций и тем самым экономить время.

→ Важно понимать, что для некоторых функций не из всех стартовых точек получается достичь сходимости метода. Тогда их можно перебирать, к примеру, с помощью цикла.

✍ Итак, мы обсудили один из самых эффективных на сегодняшний день алгоритмов — вариацию BFGS квазиньютоновских методов. Вы будете регулярно сталкиваться с этим алгоритмом при решении различных задач и при использовании библиотек для оптимизации. Так что давайте попрактикуемся: в этом юните мы посмотрели фрагмент поэтапного разбора метода BFGS для функции x** 2 - x*y + y **2 + 9 *x - 6 *y + 20 — давайте завершим начатое и найдём точку минимума ↓

**ЗАДАЧА**

Найдите точку минимума для функции x** 2 - x*y + y **2 + 9 *x - 6 *y + 20.

В качестве стартовой возьмите точку (-400, -400).

Значения координат округлите до целого числа.

In [3]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - x[0]*x[1] + x[1]**2.0 + 9.0*x[0] - 6*x[1] + 20
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2* x[1] - 6])
 
# определяем начальную точку
x_0 = [-400, -400]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 9
Решение: f([-3.99999972  1.00000028]) = -1.00000


Найдите минимум функции x**2 - 3 *x + 45 с помощью квазиньютоновского метода BFGS.

В качестве стартовой точки возьмите x = 10.

В качестве ответа введите минимальное значение функции в достигнутой точке.

In [4]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - 3*x[0] + 45
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = 10
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([1.5]) = 42.75000


Решите предыдущую задачу, применяя модификацию L-BFGS-B.

В каком случае получилось меньше итераций?

In [5]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - 3*x[0] + 45
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = 10
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 5
Решение: f([1.5]) = 42.75000


Найдите минимум функции x** 4 + 6*y**2 + 10, взяв за стартовую точку (100,100).

Какой алгоритм сошелся быстрее?

Каково минимальное значение функции в найденной точке минимума?

In [6]:
# определяем нашу функцию
def func(x):
    return x[0]**4.0 + 6.0 * (x[1]**2.0) + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4*(x[0]**3),12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Количество оценок: 40
Решение: f([-9.52718297e-03 -2.32170510e-06]) = 10.00000


In [7]:
# определяем нашу функцию
def func(x):
    return x[0]**4.0 + 6.0 * (x[1]**2.0) + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4*(x[0]**3),12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000
